<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Prompts

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// Retrieve the configuration settings for the Azure OpenAI service
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [3]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Create a new Semantic Kernel instance with the Azure OpenAI Chat Completion service
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.11.1

### Step 2 - Execute a Semantic Kernel Prompt

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper decision frames
* Gather the information & intelligence in order to make a decision
* Recommend decision frameworks to make a high-quality decision
* Provide feedback to evaluate a decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks. 

In [4]:
var simplePrompt = "Provide some decision frameworks that can help improve the quality of decisions.";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simplePrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

1. SWOT Analysis: This examines the Strengths, Weaknesses, Opportunities, and Threats related to a decision, and helps in identifying the strategic factors that can impact the decision-making process.

2. Decision Matrix: This framework helps evaluate multiple options based on specific criteria, by ranking each option based on the importance of each criterion, and then selecting the option with the highest overall score.

3. Cost-Benefit Analysis: This involves assessing the costs and benefits associated with each option, and choosing the one that offers the maximum net benefit.

4. Six Thinking Hats: This framework encourages exploring different perspectives by wearing six different "hats" – each representing a different thinking style (e.g., emotional, factual, creative). It helps in considering all aspects of a decision.

5. Pareto Analysis (80/20 Rule): This helps prioritize the factors that will have the most significant impact on the decision, focusing on the 20% of factors that 

### Step 3 - Execute a Semantic Kernel Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [5]:
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt))
{
   Console.Write(streamChunk);
}

1. Six Thinking Hats: This framework involves examining a decision from six different perspectives (emotional, informational, logical, creative, critical, and organizational) to ensure a well-rounded evaluation.

2. SWOT Analysis: This involves analyzing the Strengths, Weaknesses, Opportunities, and Threats related to a decision to identify potential risks and benefits.

3. Decision Matrix: This involves creating a matrix to evaluate different options based on multiple criteria. Each option is scored based on how well it meets each criterion, and the option with the highest total score is chosen.

4. Cost-Benefit Analysis: This involves comparing the costs and benefits of each option to determine which one offers the greatest net benefit.

5. Pareto Analysis: This technique helps prioritize options by identifying the few factors that will have the largest impact on the decision outcome.

6. Decision Tree: This involves mapping out the different choices available and the potential outco

### Step 4 - Execute a Semantic Kernel Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

In [8]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simplePrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

1. SWOT Analysis: This framework helps identify the strengths, weaknesses, opportunities, and threats related to a decision. It provides a comprehensive view of the internal and external factors that can impact the decision.

2. Decision Matrix: This framework helps evaluate and prioritize different options based on specific criteria. Each option is scored based on how well it meets each criterion, and the option with the highest score is selected.

3. Cost-Benefit Analysis: This framework helps assess the costs and benefits of different options. It involves calculating the total costs and benefits of each option and comparing them to determine which option provides the greatest net benefit.

4. Pareto Analysis: This framework helps identify the most significant factors that impact a decision. It involves ranking options based on their contribution to the overall goal and focusing on the options that provide the greatest impact.

5. Decision Tree: This framework helps visualize the dif

### Step 5 - Execute a Semantic Kernel Prompt with a system prompt (persona)

All the best practices of prompt engineering apply when using Semantic Kernel.  
* Make the prompt more specific
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Tell the AI what to do to avoid doing something wrong
* Provide context to the AI
* Using message roles in chat completion prompts
* Give your AI words of encouragement  

One best practice is to provide a common behaviour across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. These prompts can be dynamically created with a programming language like C#.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. The decision framework responses are much more robust with decision intelligence information.

In [13]:
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

var simpleDecisionPromptTemplate = $"""
{systemPrompt}

Request from the user: {simplePrompt}
""";

Console.WriteLine("Prompt To Send to the Azure OpenAI Chat Completion Service...");
Console.WriteLine(simpleDecisionPromptTemplate);

Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("Response from the Azure OpenAI Chat Completion Service...");
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

Prompt To Send to the Azure OpenAI Chat Completion Service...
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

Request from the user: Provide some decision frameworks that can help improve the quality of decisions.

----------------------------------------

Response from the Azure OpenAI Chat Completion Service...
Certainly! Making high-quality decisions is crucial in both personal and professional contexts. Here are several decision frameworks that can help improve the quality of your decisions:

1. **SWOT Analysis**:
   - **Strengths**: Identify what advantages you have in the current situation.
   - **Weaknesses**: Recognize the areas where you or your organization may be lacking.
   - **Opportunities**: Look for external factors that could be leveraged for success.
   - **Threats**: Be aware of ex